In [99]:
import onnxruntime
import numpy as np
from PIL import Image 
import os
import time
import cv2

In [85]:
# labels = ["beige","black","blue","brown","gold","green","grey","orange","pink","purple","red","silver","tan","white","yellow"]
labels = ['black', 'blue', 'brown', 'green', 'orange', 'pink', 'red', 'silver', 'white', 'yellow']

In [86]:
session = onnxruntime.InferenceSession("./models/models_onnx/model_color.onnx",providers=['CUDAExecutionProvider'])
session.get_inputs()[0].shape
session.get_inputs()[0].type

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [100]:

#OpenCv
def predict(img_path):
    #img = Image.open(img_path)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    img = cv2.resize(img,(224, 224))
    img_array = np.array(img)
    print( f'{img_array.shape[0]} {img_array.shape[1]} {img_array.shape[2]}') 

    img_array = img_array.astype('float32') / 255.0 # Normalize pixel values to [0, 1]
    # for i in range(img_array.shape[0]):
    #     for j in range(img_array.shape[1]):
    #         for k in range(img_array.shape[2]):
    #             print(img_array[i, j, k], end=' ')
    #         print()
    #     print()
    img_array = np.expand_dims(img_array, axis=0)
  
    start_time = time.time()
    predictions = session.run([output_name], {input_name: img_array})
    end_time = time.time()
    total_time =  (end_time - start_time) * 1000  # Convert to milliseconds
    # print(total_time)
    predicted_class_index = np.argmax(predictions[0])
    print(predictions[0][0][predicted_class_index])
    return labels[predicted_class_index],total_time

In [50]:
#PIL
def predict(img_path):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = img_array.astype('float32')/ 255.0 # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0) 
    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]):
            for k in range(img_array.shape[2]):
                print(img_array[i, j, k], end=' ')
            print()
        print()
    start_time = time.time()
    predictions = session.run([output_name], {input_name: img_array})
    end_time = time.time()
    total_time = (end_time - start_time) * 1000  # Convert to milliseconds
    print(total_time)
    predicted_class_index = np.argmax(predictions[0])
    print(predictions[0][0][predicted_class_index])
    
    return labels[predicted_class_index]

In [83]:
print(predict('red.jpg'))

224 224 3
0.9101917
orange


In [ ]:
exp_th = 8
images_path = 'D:/PROJECTS/AI/Color/Data_vehicle/images_test/test'
images = os.listdir(images_path)
save_path = 'D:/PROJECTS/AI/Color/Data_vehicle/images_test/runs'
save_path += f'/exp_onnx{exp_th}'
i = 0
total_time = 0
for image in images:
    color,time_t = predict(os.path.join(images_path, image))
    total_time += time_t
    folder_save = os.path.join(save_path, color)
    if not os.path.exists(folder_save):
        os.makedirs(folder_save)
    Image.open(os.path.join(images_path, image)).save(os.path.join(folder_save, image))
    i += 1
    if i == 2000:
        break
print(total_time/2000)